# Para este proyecto, busco extraer los datos del producto y del precio del mismo de la pagina web de Hiper Libertad:
 https://www.hiperlibertad.com.ar/

## 00 - Instalar librerias necesarias:

* Scrapy: para definir "arañas" (spiders) que navegan por sitios web y extraen información estructurad
* Datetime: para definir fecha y hora

In [59]:
# 00- Instalar librerias necesarias:
!pip install scrapy
!pip install datetime

## 01 -  Importar las librerias necesarias:

* scrapy: para seleccionar elementos HTML con selectores CSS
* datetime: para manejar las fechas y horas
* pytz: para trabajar con zonas horarias
* google.colab.files: para subir o descargar archivos desde colab a la computadora

In [60]:
import scrapy
from datetime import datetime
import pytz
from google.colab import files

## 02 -  Desarrollar y guardar el script en el archivo hiper_libertad.py

Se define la araña web y una lista de URLs que corresponden a productos del supermercado Hiper Libertad. Asi mismo, se define la fundion parse, que procesa cada pagina

In [61]:
%%writefile hiper_libertad.py

# 01 - Importo librerias necesarias:
import scrapy
from scrapy.cmdline import execute
from datetime import datetime
import pytz


# 02-  Configuro el la fecha actual:
zona_arg = pytz.timezone('America/Argentina/Buenos_Aires')

hora = datetime.now(zona_arg).time()
fecha = datetime.now().today()

#03- Formato correcto para la fecha y hora:
fHoraMinuto = "%H:%M"
fDiaMesAño = "%d-%m-%Y"

horaActual = hora.strftime(fHoraMinuto)
fechaActual = fecha.strftime(fDiaMesAño)

# prueba de resultado:
#print(f'''
#La fecha actual es: {fechaActual}
#La hora actual es: {horaActual}''')

# 04 - Configuro la araña:
class ProductosSpider(scrapy.Spider):
    name = 'Canasta'
    start_urls = [
        "https://www.hiperlibertad.com.ar/pan-frances-x-500-g/p",
        "https://www.hiperlibertad.com.ar/galletita-cracker-la-providencia-clasica-x303g/p",
        "https://www.hiperlibertad.com.ar/galletas-terrabusi-variedad-dorada-x-300-gr/p",
        "https://www.hiperlibertad.com.ar/arroz-parboil-taeq-240gr/p",
        "https://www.hiperlibertad.com.ar/harina-de-trigo-morixe-000-1-kg/p",
        "https://www.hiperlibertad.com.ar/fideos-tallarines-n7-matarazzo-x-500-gr/p",
        "https://www.hiperlibertad.com.ar/papa-cepillada-x-1-kg/p",
        "https://www.hiperlibertad.com.ar/batata-x-500-g/p",
        "https://www.hiperlibertad.com.ar/azucar-azucel-1-kilo/p",
        "https://www.hiperlibertad.com.ar/mermelada-dulcor-durazno-frasco-454-gr/p",
        "https://www.hiperlibertad.com.ar/lentejas-taeq-200gr/p",
        "https://www.hiperlibertad.com.ar/arvejas-montenevi-x-340-gr/p",
        "https://www.hiperlibertad.com.ar/acelga-congelada-green-life-500-g/p",
        "https://www.hiperlibertad.com.ar/cebolla-x-500-g/p",
        "https://www.hiperlibertad.com.ar/bandeja-lechuga-mantecosa-lavada-sue-o-verde-x-200-gr/p",
        "https://www.hiperlibertad.com.ar/tomate-perita-x-500g/p",
        "https://www.hiperlibertad.com.ar/calabacin-anquito-fraccionado-x-700gr-1-2-calabacin/p",
        "https://www.hiperlibertad.com.ar/mandarina-x-1kg/p",
        "https://www.hiperlibertad.com.ar/naranja-p-jugo-x-1kg/p",
        "https://www.hiperlibertad.com.ar/banana-x-kg-ecomm/p",
        "https://www.hiperlibertad.com.ar/pollo-fresco-x-un-aprox-2-5kg/p",
        "https://www.hiperlibertad.com.ar/pulpa-de-jamon-de-cerdo-x-500-g/p",
        "https://www.hiperlibertad.com.ar/milanesa-de-bola-de-lomo-de-novillito-x-500-g/p",
        "https://www.hiperlibertad.com.ar/huevo-blanco-carnave-carton-x-12-unidades/p",
        "https://www.hiperlibertad.com.ar/leche-entera-c-calcio-la-lacteo-larga-vida-1-l/p",
        "https://www.hiperlibertad.com.ar/manteca-tonadita-200gr/p",
        "https://www.hiperlibertad.com.ar/aceite-de-girasol-alsamar-x-900-ml/p",
        "https://www.hiperlibertad.com.ar/brahma-chopp-can-4x6-473cc/p",
        "https://www.hiperlibertad.com.ar/sal-fina-dos-anclas-250-gr/p",
        "https://www.hiperlibertad.com.ar/mayonesa-hellmanns-clasica-x-475-gr/p",
        "https://www.hiperlibertad.com.ar/cafe-molido-bonafide-sensaciones-torrado-intenso-500-gr/p",
        "https://www.hiperlibertad.com.ar/yerba-playadito-suave-bcp-1kg/p",
    ]

# 05 - Configuro la funcion Parse:

    def parse(self, response):
            nombre_producto = response.css('span.vtex-store-components-3-x-productBrand::text').get()

            # El precio esta dividido en 2 span diferentes (enteros y decimales)
            precio_entero = response.css("span.vtex-product-price-1-x-currencyInteger::text").getall()
            precio_decimales =  response.css("span.vtex-product-price-1-x-currencyFraction::text").get()

            if precio_entero: # Si tiene un precio configurado
              if len(precio_entero) == 1:  # Tiene solamente un preciosin ofertas
                precio = f'{precio_entero[0]}, {precio_decimales}'
              elif len(precio_entero[0])> 1: # cuando el producto tiene un precio anterior, enfatizando una oferta
                precio = f'{precio_entero[1]}, {precio_decimales[1]}'  # Se toma el segundo valor, que es el precio real sin promocion.
              else:
                precio = f'{precio_entero[0]}.{precio_entero[1]}, {precio_decimales}' # cuando tiene entero y decimales
            else: # Si no tiene un precion configurado
              precio = "No disponible"

            yield {
                "fecha": fechaActual,
                "hora": horaActual,
                "Nombre Producto": nombre_producto,
                "Precio": precio,
                "URL": response.url
            }

Overwriting hiper_libertad.py


## 03 - Ejecutar el scrapper
Se ejecuta el scraper en el archivo hiper_libertad.py y guarda los resultados en un archivo CSV, llamado relevamiento_precios_hiper_libertad.csv.

In [62]:
# Ejecutar:
!scrapy runspider hiper_libertad.py -o relevamiento_precios_hiper_libertad.csv


2025-04-10 23:12:32 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
2025-04-10 23:12:32 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.12.9, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2025-04-10 23:12:32 [scrapy.addons] INFO: Enabled addons:
[]
2025-04-10 23:12:32 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2025-04-10 23:12:32 [scrapy.extensions.telnet] INFO: Telnet Password: e30091c95156efbe
2025-04-10 23:12:32 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2025-04-10 23:12:32 [scrapy.crawler] INFO: Overridden settings:
{'S

## 04 - Descargar el archivo a la PC:

In [63]:
files.download('relevamiento_precios_hiper_libertad.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>